# Using Delta Lake with Daft

[Daft](www.getdaft.io) is a framework for ETL, analytics, and ML/AI at scale with a familiar Python dataframe API, implemented in Rust. 

For Delta Lake users, Daft is a great data processing tool because it gives you:
- **Skipping Filtered Data**: Daft implements automatic partition pruning and stats-based file pruning for filter predicates, skipping data that doesn’t need to be read.
- **Multimodal Dataframes**: read, write and transform multimodal data incl. images, JSON, PDF, audio, etc. 
- **Parallel + Distributed Reads**: Daft parallelizes Delta Lake table reads over all cores of your machine, if using the default multithreading runner, or all cores + machines of your Ray cluster, if using the distributed Ray runner.
- **Multi-cloud Support**: Daft supports reading Delta Lake tables from AWS S3, Azure Blob Store, and GCS, as well as local files.

Daft and Delta Lake work really well together. Daft provides unified compute for Delta Lake’s unified storage. Together, Delta Lake and Daft give you high-performance query optimization and distributed compute on massive datasets.

## Installing Daft for Delta Lake

The easiest way to use Delta Lake format with Daft DataFrames is to install Daft with the `[deltalake]` extras using `pip`:

In [ ]:
!pip install -U "getdaft[deltalake]"

This adds the `deltalake` Python package to your install. This package is used to fetch metadata about the Delta Lake table, such as paths to the underlying Parquet files and table statistics. You can of course also install the `deltalake` manually.

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Read Delta Lake into a Daft DataFrame

You can easily read Delta Lake tables into a Daft DataFrame using the `read_delta_lake` method. Let's use it to read some data stored in a Delta Lake on disk:

In [2]:
import daft

# read delta table into Daft DataFrame
delta_path = "../../data/people_countries_delta_dask"
df = daft.read_delta_lake(delta_path)

You can also read in Delta Lake data from remote sources like S3:

In [3]:
# table_uri = (
#     "s3://daft-public-datasets/red-pajamas/"
#     "stackexchange-sample-north-germanic-deltalake"
# )
# df = daft.read_delta_lake(table_uri)

In [4]:
df

first_nameUtf8,last_nameUtf8,countryUtf8,continentUtf8


Daft DataFrames are lazy by default. This means that the contents will not be computed ("materialized") unless you explicitly tell Daft to do so. This is best practice for working with larger-than-memory datasets and parallel/distributed architectures.

The Delta table we have just loaded only has 5 rows. You can materialize it in memory using the `.collect` method:

In [5]:
df.collect()

first_nameUtf8,last_nameUtf8,countryUtf8,continentUtf8
Ernesto,Guevara,Argentina,NaN
Bruce,Lee,China,Asia
Jack,Ma,China,Asia
Wolfgang,Manche,Germany,NaN
Soraya,Jala,Germany,NaN


## What can I do with a Daft DataFrame?

Daft gives you [full-featured DataFrame functionality](https://www.getdaft.io/projects/docs/en/latest/user_guide/basic_concepts.html), similar to what you might be used to from pandas, Dask or PySpark. 

On top of this, Daft also gives you:
- **Multimodal data type support** to work with Images, URLs, Tensors and more
- **Expressions API** for easy column transformations
- **UDFs** for multi-column transformation, incl. ML applications


Let's take a quick look at some of Daft's basic DataFrame operations.

You can **select columns** from your DataFrame using the `select` method. We'll use the `show` method to show the first `n` rows (defaults to 10):

In [6]:
df.select("first_name", "country").show()

first_nameUtf8,countryUtf8
Ernesto,Argentina
Bruce,China
Jack,China
Wolfgang,Germany
Soraya,Germany


You can also **index** the DataFrame with a list of columns as a useful alias for `select` operations:

In [7]:
df[["first_name", "country"]].show()

first_nameUtf8,countryUtf8
Ernesto,Argentina
Bruce,China
Jack,China
Wolfgang,Germany
Soraya,Germany


You can **sort** your Daft DataFrame using the `sort` method:

In [8]:
df.sort(df["country"], desc=True).show()

first_nameUtf8,last_nameUtf8,countryUtf8,continentUtf8
Wolfgang,Manche,Germany,NaN
Soraya,Jala,Germany,NaN
Bruce,Lee,China,Asia
Jack,Ma,China,Asia
Ernesto,Guevara,Argentina,NaN


You can **filter** your DataFrame using the `where` method:

In [9]:
df.where(df["continent"] == "Asia").show()

first_nameUtf8,last_nameUtf8,countryUtf8,continentUtf8
Bruce,Lee,China,Asia
Jack,Ma,China,Asia


You can group your DataFrame by a specific columns using the `groupby` method. You can then specify the aggregation method, in this case using the `count` aggregator:

In [10]:
df.select("first_name", "country").groupby(df["country"]).count("first_name").show()

countryUtf8,first_nameUInt64
Germany,2
China,2
Argentina,1


Check out the [Daft User Guide](https://www.getdaft.io/projects/docs/en/latest/user_guide/index.html) for a complete list of DataFrame operations. 

## Data Skipping Optimizations

You may have noticed the Delta Lake warning at the top when we first called `collect` on our DataFrame: 

<div class="alert alert-block alert-danger">WARNING: has partitioning keys = [PartitionField(country#Utf8)], but no partition filter was specified. This will result in a full table scan.</div>

Delta Lake is informing us that the data is partitioned on the `country` column. 

Daft's native query optimizer has access to all of the Delta Lake metadata. This means it can optimize your query by skipping the partitions that are not relevant for this query. Instead of having to read all 3 partitions, we can read only 1 and get the same result, just faster!

In [12]:
# Filter on partition columns will result in efficient partition pruning; non-matching partitions will be skipped.
df.where(df["country"] == "Germany").show()

first_nameUtf8,last_nameUtf8,countryUtf8,continentUtf8
Wolfgang,Manche,Germany,NaN
Soraya,Jala,Germany,NaN


In [13]:
# It would be awesome to have some evidence to back up the claim that this query with partition filter is faster.
# Below I run explain(True) with a filter on a partition vs a non-partition column. I'd expect a difference (in favor of country==Germany)
# But AFAICT the plans are more or less identical

In [11]:
df.where(df["continent"] == "Asia").explain(True)

== Unoptimized Logical Plan ==

* Filter: col(continent) == lit("Asia")
|
* Source:
|   Number of partitions = 3
|   Output schema = first_name#Utf8, last_name#Utf8, country#Utf8, continent#Utf8


== Optimized Logical Plan ==

* Filter: col(continent) == lit("Asia")
|
* Source:
|   Number of partitions = 3
|   Output schema = first_name#Utf8, last_name#Utf8, country#Utf8, continent#Utf8


== Physical Plan ==

* Filter: col(continent) == lit("Asia")
|
* InMemoryScan:
|   Schema = first_name#Utf8, last_name#Utf8, country#Utf8, continent#Utf8
|   Size bytes = 358
|   Clustering spec = { Num partitions = 3 }



In [12]:
df.where(df["country"] == "Germany").explain(True)

== Unoptimized Logical Plan ==

* Filter: col(country) == lit("Germany")
|
* Source:
|   Number of partitions = 3
|   Output schema = first_name#Utf8, last_name#Utf8, country#Utf8, continent#Utf8


== Optimized Logical Plan ==

* Filter: col(country) == lit("Germany")
|
* Source:
|   Number of partitions = 3
|   Output schema = first_name#Utf8, last_name#Utf8, country#Utf8, continent#Utf8


== Physical Plan ==

* Filter: col(country) == lit("Germany")
|
* InMemoryScan:
|   Schema = first_name#Utf8, last_name#Utf8, country#Utf8, continent#Utf8
|   Size bytes = 358
|   Clustering spec = { Num partitions = 3 }



Read [High-Performance Querying on Massive Delta Lake Tables with Daft](link) for an in-depth benchmarking of query optimization with Delta Lake and Daft.

## Transform columns with Expressions

Daft provides a flexible [Expressions](https://www.getdaft.io/projects/docs/en/latest/api_docs/expressions.html) API for defining computation that needs to happen over your columns.

For example, we can use `daft.col()` expressions together with the `with_column` method to create a new column `full_name`, joining the contents of the `last_name` column to the `first_name` column:

In [21]:
df_full = df.with_column("full_name", daft.col('first_name') + ' ' + daft.col('last_name'))
df_full.show()

first_nameUtf8,last_nameUtf8,countryUtf8,continentUtf8,full_nameUtf8
Ernesto,Guevara,Argentina,NaN,Ernesto Guevara
Bruce,Lee,China,Asia,Bruce Lee
Jack,Ma,China,Asia,Jack Ma
Wolfgang,Manche,Germany,NaN,Wolfgang Manche
Soraya,Jala,Germany,NaN,Soraya Jala


## Multimodal Data Type Support

Daft has a rich multimodal type-system with support for Images, URLs, Tensors and more.

Daft columns can contain any Python objects. For example, let's add a column containing a Python class `Dog` for some of the people in our dataset:

In [22]:
import numpy as np

class Dog:
    def __init__(self, name):
        self.name = name
        
    def bark(self):
        return f"{self.name}!"

df_dogs = daft.from_pydict({
    'full_name': ['Ernesto Guevara','Bruce Lee','Jack Ma','Wolfgang Manche','Soraya Jala'],
    "dogs": [Dog("ruffles"), Dog("shnoodles"), Dog("waffles"), Dog("doofus"), Dog("Fluffles")],
})

df_dogs.show()

full_nameUtf8,dogsPython
Ernesto Guevara,<__main__.Dog object at 0x13e063dd0>
Bruce Lee,<__main__.Dog object at 0x11a64bf10>
Jack Ma,<__main__.Dog object at 0x13e514350>
Wolfgang Manche,<__main__.Dog object at 0x13e514690>
Soraya Jala,<__main__.Dog object at 0x13e517310>


You can join this new `dogs` column to your existing DataFrame using the `join` method:

In [23]:
df_family = df_full.join(df_dogs, on=["full_name"])
df_family.show()

full_nameUtf8,first_nameUtf8,last_nameUtf8,countryUtf8,continentUtf8,dogsPython
Ernesto Guevara,Ernesto,Guevara,Argentina,NaN,<__main__.Dog object at 0x13e063dd0>
Bruce Lee,Bruce,Lee,China,Asia,<__main__.Dog object at 0x11a64bf10>
Jack Ma,Jack,Ma,China,Asia,<__main__.Dog object at 0x13e514350>
Wolfgang Manche,Wolfgang,Manche,Germany,NaN,<__main__.Dog object at 0x13e514690>
Soraya Jala,Soraya,Jala,Germany,NaN,<__main__.Dog object at 0x13e517310>


We can then use the `apply` method to apply a function to each instance of the Dog class:

In [24]:
from daft import DataType

df_family = df_family.with_column(
    "dogs_bark_name",
    df_family["dogs"].apply(lambda dog: dog.bark(), return_dtype=DataType.string()),
)

df_family.show()

first_nameUtf8,last_nameUtf8,countryUtf8,continentUtf8,full_nameUtf8,dogsPython,dogs_bark_nameUtf8
Ernesto,Guevara,Argentina,NaN,Ernesto Guevara,<__main__.Dog object at 0x13e063dd0>,ruffles!
Bruce,Lee,China,Asia,Bruce Lee,<__main__.Dog object at 0x11a64bf10>,shnoodles!
Jack,Ma,China,Asia,Jack Ma,<__main__.Dog object at 0x13e514350>,waffles!
Wolfgang,Manche,Germany,NaN,Wolfgang Manche,<__main__.Dog object at 0x13e514690>,doofus!
Soraya,Jala,Germany,NaN,Soraya Jala,<__main__.Dog object at 0x13e517310>,Fluffles!


Daft DataFrames can also contain [many other data types](https://www.getdaft.io/projects/docs/en/latest/api_docs/datatype.html), like tensors, JSON, URLs and images. The Expressions API provides useful tools to work with these data types. 

For example, let's say we have images of our 5 dogs stored at a URL. We can add these URLs to our DataFrame and download the bytes into a new column using the `url.download()` Expression:

In [25]:
df_dogs_img = daft.from_pydict({
    "urls": [
        "https://i.natgeofe.com/n/4f5aaece-3300-41a4-b2a8-ed2708a0a27c/domestic-dog_thumb_square.jpg?wp=1&w=272&h=272",
        "https://i.pinimg.com/736x/ab/e5/5f/abe55ff1a275ae726657fb1d03a37a97.jpg",
        "https://kb.rspca.org.au/wp-content/uploads/2018/11/golder-retriever-puppy.jpeg",
        "https://www.thesprucepets.com/thmb/hxWjs7evF2hP1Fb1c1HAvRi_Rw0=/2765x0/filters:no_upscale():strip_icc()/chinese-dog-breeds-4797219-hero-2a1e9c5ed2c54d00aef75b05c5db399c.jpg",
        "https://www.dogstrust.org.uk/images/800x600/assets/2023-12/Web%20header%20image-dexter_outdoors_kenilworth_dogstrust.jpg"
    ],
    "full_name": ['Ernesto Guevara','Bruce Lee','Jack Ma','Wolfgang Manche','Soraya Jala']
})
df_dogs_img = df_dogs_img.with_column("image_bytes", df_dogs_img["urls"].url.download())
df_dogs_img.show()

urlsUtf8,full_nameUtf8,image_bytesBinary
https://i.natgeofe.com/n/4f5aaece-3300-41a4-b2a8-ed2708a0a27c/domestic-dog_thumb_square.jpg?wp=1&w=272&h=272,Ernesto Guevara,"b""\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01""..."
https://i.pinimg.com/736x/ab/e5/5f/abe55ff1a275ae726657fb1d03a37a97.jpg,Bruce Lee,"b""\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01""..."
https://kb.rspca.org.au/wp-content/uploads/2018/11/golder-retriever-puppy.jpeg,Jack Ma,"b""\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01""..."
https://www.thesprucepets.com/thmb/hxWjs7evF2hP1Fb1c1HAvRi_Rw0=/2765x0/filters:no_upscale():strip_icc()/chinese-dog-breeds-4797219-hero-2a1e9c5ed2c54d00aef75b05c5db399c.jpg,Wolfgang Manche,"b""\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01""..."
https://www.dogstrust.org.uk/images/800x600/assets/2023-12/Web%20header%20image-dexter_outdoors_kenilworth_dogstrust.jpg,Soraya Jala,"b""\xff\xd8\xff\xdb\x00C\x00\x06\x04\x05""..."


We can then apply the `image.decode()` Expression to decode the bytes into human-readable images, right within our Jupyter notebook:

In [26]:
df_dogs_img = df_dogs_img.with_column("image", daft.col("image_bytes").image.decode())

In [27]:
df_dogs_img.show()

╭────────────────────────────────┬─────────────────┬────────────────────────────────┬──────────────╮
│ urls                           ┆ full_name       ┆ image_bytes                    ┆ image        │
│ ---                            ┆ ---             ┆ ---                            ┆ ---          │
│ Utf8                           ┆ Utf8            ┆ Binary                         ┆ Image[MIXED] │
╞════════════════════════════════╪═════════════════╪════════════════════════════════╪══════════════╡
│ https://i.natgeofe.com/n/4f5a… ┆ Ernesto Guevara ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://i.pinimg.com/736x/ab/… ┆ Bruce Lee       ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://kb.rspca.org.au/wp-co… ┆ Jack Ma         ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.thesprucepets.com… ┆ Wolfgang Manche ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ https://www.dogstrust.org.uk/… ┆ Soraya Jala     ┆ b"\xff\xd8\xff\xdb\x00C\x00\x… ┆ <Image>      │
╰────────────────────────────────┴─────────────────┴────────────────────────────────┴──────────────╯

(Showing first 5 of 5 rows)

Finally, we can `join` on `full_name` and `owners` to bring these beautiful fluffy beings back to their owners:

In [28]:
df_family = df_family.join(df_dogs_img.select("full_name", "image"), on=["full_name"])

In [29]:
df_family.show()

╭─────────────────┬────────────┬───────────┬────────────┬─────────────────────────┬────────────────┬──────────────╮
│ full_name       ┆ first_name ┆ last_name ┆      …     ┆ dogs                    ┆ dogs_bark_name ┆ image        │
│ ---             ┆ ---        ┆ ---       ┆            ┆ ---                     ┆ ---            ┆ ---          │
│ Utf8            ┆ Utf8       ┆ Utf8      ┆ (2 hidden) ┆ Python                  ┆ Utf8           ┆ Image[MIXED] │
╞═════════════════╪════════════╪═══════════╪════════════╪═════════════════════════╪════════════════╪══════════════╡
│ Ernesto Guevara ┆ Ernesto    ┆ Guevara   ┆ …          ┆ <__main__.Dog object at ┆ ruffles!       ┆ <Image>      │
│                 ┆            ┆           ┆            ┆ 0x13e…                  ┆                ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Bruce Lee       ┆ Bruce      ┆ Lee       ┆ …          ┆ <__main__.Dog object at ┆ shnoodles!     ┆ <Image>      │
│                 ┆            ┆           ┆            ┆ 0x11a…                  ┆                ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Jack Ma         ┆ Jack       ┆ Ma        ┆ …          ┆ <__main__.Dog object at ┆ waffles!       ┆ <Image>      │
│                 ┆            ┆           ┆            ┆ 0x13e…                  ┆                ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Wolfgang Manche ┆ Wolfgang   ┆ Manche    ┆ …          ┆ <__main__.Dog object at ┆ doofus!        ┆ <Image>      │
│                 ┆            ┆           ┆            ┆ 0x13e…                  ┆                ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Soraya Jala     ┆ Soraya     ┆ Jala      ┆ …          ┆ <__main__.Dog object at ┆ Fluffles!      ┆ <Image>      │
│                 ┆            ┆           ┆            ┆ 0x13e…                  ┆                ┆              │
╰─────────────────┴────────────┴───────────┴────────────┴─────────────────────────┴────────────────┴──────────────╯

(Showing first 5 of 5 rows)

## WOOF! 🐶 🥳

For a full list of all Expression methods and operators, see the [Expressions API Docs](https://www.getdaft.io/projects/docs/en/latest/api_docs/expressions.html#api-numeric-expression-operations).

## Transform multiple columns with UDFs

You can use User-Defined Functions (UDFs) to run functions over multiple rows or columns:

In [30]:
from daft import udf

@udf(return_dtype=DataType.string())
def custom_bark(dog_series, owner_series):
    return [
        f"{dog.name} loves {owner_name}!"
        for dog, owner_name
        in zip(dog_series.to_pylist(), owner_series.to_pylist())
    ]

df_family.with_column("custom_bark", custom_bark(df_family["dogs"], df_family["first_name"])).show()

╭────────────┬───────────┬───────────┬────────────┬────────────────┬──────────────┬────────────────────────╮
│ first_name ┆ last_name ┆ country   ┆      …     ┆ dogs_bark_name ┆ image        ┆ custom_bark            │
│ ---        ┆ ---       ┆ ---       ┆            ┆ ---            ┆ ---          ┆ ---                    │
│ Utf8       ┆ Utf8      ┆ Utf8      ┆ (3 hidden) ┆ Utf8           ┆ Image[MIXED] ┆ Utf8                   │
╞════════════╪═══════════╪═══════════╪════════════╪════════════════╪══════════════╪════════════════════════╡
│ Ernesto    ┆ Guevara   ┆ Argentina ┆ …          ┆ ruffles!       ┆ <Image>      ┆ ruffles loves Ernesto! │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Bruce      ┆ Lee       ┆ China     ┆ …          ┆ shnoodles!     ┆ <Image>      ┆ shnoodles loves Bruce! │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Jack       ┆ Ma        ┆ China     ┆ …          ┆ waffles!       ┆ <Image>      ┆ waffles loves Jack!    │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Wolfgang   ┆ Manche    ┆ Germany   ┆ …          ┆ doofus!        ┆ <Image>      ┆ doofus loves Wolfgang! │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Soraya     ┆ Jala      ┆ Germany   ┆ …          ┆ Fluffles!      ┆ <Image>      ┆ Fluffles loves Soraya! │
╰────────────┴───────────┴───────────┴────────────┴────────────────┴──────────────┴────────────────────────╯

(Showing first 5 of 5 rows)

Daft supports workloads with [many more data types](https://www.getdaft.io/projects/docs/en/latest/api_docs/datatype.html) than traditional DataFrame APIs. 

By combining multimodal data support with the UDF functionality you can [run ML workloads](https://www.getdaft.io/projects/docs/en/latest/user_guide/tutorials.html#mnist-digit-classification) right within your DataFrame.

## When should I use Daft DataFrames?

Daft DataFrames are designed for multimodal, distributed workloads. 

You may want to consider using Daft if you're working with:

1. **Large datasets** that don't fit into memory or would benefit from parallelization
2. **Multimodal data types**, such as images, JSON, vector embeddings, and tensors
3. **ML workloads** that would benefit from interactive computation within DataFrame (via UDFs)
4 ...

Take a look at the [Daft tutorials](https://www.getdaft.io/projects/docs/en/latest/user_guide/tutorials.html) for in-depth examples of each use case.

## Contribute to `daft`

Excited about Daft and want to contribute? Please join us on [Github](https://github.com/Eventual-Inc/Daft) 🚀